In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("en/train", delim_whitespace=True, names = ["Word", "Tag"])
df.head(60)

,Word,Tag
0,Municipal,B-NP
1,bonds,I-NP
2,are,B-VP
3,generally,B-ADVP
4,a,B-ADJP
5,bit,I-ADJP
6,safer,I-ADJP
7,than,B-PP
8,corporate,B-NP
9,bonds,I-NP


In [51]:
df.isnull().values.any()

False

In [54]:
tag_word_table = {}
df_truncate = df[:20]
for i in range(len(df_truncate)):
    line = df_truncate.iloc[i]
    tag = line["Tag"]
    word = line["Word"]
    if tag in tag_word_table:
        tag_word_table[tag].append(word)
    else:
        tag_word_table[tag] = [word]

print(tag_word_table)

{'B-NP': ['Municipal', 'corporate', 'a', 'bonds'], 'I-NP': ['bonds', 'bonds', 'recession'], 'B-VP': ['are'], 'B-ADVP': ['generally'], 'B-ADJP': ['a', 'not'], 'I-ADJP': ['bit', 'safer', 'as', 'safe'], 'B-PP': ['than', 'in', 'as'], 'O': [',', 'but']}


In [63]:
def emission(x,y, k = 0.5):
    word_list = tag_word_table[y]
    if x == "#UNK#":
        emission_count = k
    else:
        emission_count = word_list.count(x)
    ycount = len(word_list)
    return emission_count / (ycount + k)

In [64]:
emission("bonds", "I-NP")

0.5714285714285714

In [66]:
emission("#UNK#", "B-NP")

0.1111111111111111